"""
The purpose of this script is to automate SLA tasks. This includes creating folders for each month and establishment,
creating emails and letters from templates, and more.   a


Process:
1) Copy list of items from online agenda into text doc as shown below
2) I will likely want to quickly remove certain text from this, such as section descriptions like 'New Liquor License Applications,' or other info that will not go into the final outputs like the description of the license type. 
3) The most important thing will likely be keeping each entry to one line in the input file as I will likely use line breaks to parse each item. 
4) If this doesn't work, another option might be the period (.) followed by blank spaces. 
5) DBA format seems to be in parentheses after the name. 
6) I will have to manually add lawyers names.
7) Agenda number is the number in the list itself. 

----------------------------------------------------------------------------
1 - Open and read lines of text file
    - https://www.geeksforgeeks.org/reading-writing-text-files-python/
2 - Iterate through each line and build a dataframe / table from the relevant contents
3 - Filter out rows that begin with numeric. These should be the agenda items. 
4 - split at period and get first item, which is the numeric agenda item number. 
5 - split again and get the business name
6 - split again and get the address

---------------------------------------------------------------------------------

TO DO
 - Create code to remove illegal characters from strings
 - Start scripting work to input values into template letters
 - Create output for excel, which can serve a number of functions:
     - a place to input additional info like lawyers name
     - track other SLA items
 - Create similar script that takes as an input an excel file instead of a text file. 


"""

In [1]:
# Print Conda Env info running in Jupyter Notebook 
#!conda info
#import sys; print(sys.executable)

In [2]:
import pandas as pd
import datetime as dt
from datetime import datetime
import openpyxl
import os
import csv

In [3]:
agenda_pull = r"C:\Users\MN03\Desktop\Current Items\SLA_Agenda\sla_oct.txt"

In [4]:
def make_sla_folders(agenda_pull):
    
    # Open the text file
    #r+ = read/write access mode
    agenda = open(agenda_pull, 'r+')
    
    #Readlines creates a list, where each index contains a line of text, which in this case is a single establishment. 
    contents = agenda.readlines()
    
    # Close the text file
    agenda.close()
    
    # This creates a dataframe where each row is a line from the agenda pull
    agenda_df = pd.DataFrame(contents, columns=['line'])
    
    #Remove illegal character (apostrophe) from each string, and replace 'B'way' with 'Broadway'
    agenda_df = agenda_df.apply(lambda x: x.str.replace("B'way", "Broadway"))
    agenda_df = agenda_df.apply(lambda x: x.str.replace("'", ""))
    
    # This line creates new column that contains a bool series with 'True' for every line that starts with a a digit
    # Lines that do not contain a digit (agenda number) will be removed.
    agenda_df['entry_row']= agenda_df['line'].str[0].str.isdigit()
    
    # Filters out rows that do not contain an agenda item (start with a digit). 
    # Because the entry_row column is boolean, just calling it as as a filter will remove False entries.  
    agenda_df = agenda_df[agenda_df.entry_row]
    
    # Create new column with agenda number only for each row
    agenda_df['agenda_number'] = agenda_df.loc[:,'line'].str.split(pat=".").str[0]
    
    # Create column with all agenda item info except for the agenda item number
    agenda_df['agenda_info_no_number'] = agenda_df.loc[:,'line'].str.split(pat=".").str[1]
    
    # This partitions the string into:
    # 1: business name
    # 2: comma
    # 3: address, and license type and notes
    # Once partitioned the 1st and 3rd columns are output. 
    agenda_df['b_name'] = agenda_df.loc[:,'agenda_info_no_number'].str.partition(sep=",", expand=True)[0]
    agenda_df['address'] = agenda_df.loc[:,'agenda_info_no_number'].str.partition(sep=",", expand=True)[2]
    
    # This creates a column showing the text in the first parentheses. The second set is not important because these will always be
    # notes on the liquor licence, which aren't important for this exercise. 
    agenda_df['address_sup'] = agenda_df['address'].str.extract('\(([^)]+)')

    # The first set of parentheses contains either an address supplement, such as 'basement',
    # or information about the liquor license. 
    # Strings containing 'op' or 'wb' are filtered out in the code below so that only address supplementary info remains. 
    agenda_df['address_sup3'] = agenda_df['address_sup'].str.contains('op|wb', na=True)
    
    # Replace all values in the address supplement column identified above as having 'op' or 'wb' with an empty string.
    agenda_df['address_sup'] = agenda_df['address_sup'].mask(agenda_df['address_sup3'], "")
    
    # This line splits out the address and creates a new string with everything to the left of the first 
    # opening parenthesis, which is the primary address. 
    agenda_df['prim_address'] = agenda_df['address'].str.split(pat="(").str[0]
    
    # This line removes '\n' characters from each row
    #df = df.replace('\n','', regex=True)
    agenda_df['prim_address'] = agenda_df['prim_address'].replace('\n','', regex=True)
    
    # This line splits out the business name and creates a new string with everything to the left of the first 
    # opening parenthesis, which is the business trade name.
    agenda_df['b_tradename'] = agenda_df['b_name'].str.split(pat="(").str[0]
    
    # This creates a column showing the text in the first parentheses. The second set is not important because these will always be
    # notes on the liquor licence, which aren't important for this exercise. 
    agenda_df['b_llc_name'] = agenda_df['b_name'].str.extract('\(([^)]+)')

    # This replaces NAN with empty string
    agenda_df['b_llc_name'] =agenda_df['b_llc_name'].fillna('')
    
    # Current month (number and name) and year. This will be used to create top level folder. 
    month_name = str(datetime.now().strftime("%B"))
    month_num = str(datetime.now().month)
    year = str(datetime.now().year)
    
    # This line creates the top level directory with the month, year, and 'SLA'
    month_dir = month_num + '-' + month_name + ' ' + year + ' SLA'

    
    #Print message that script is running
    print("Making SLA folders at following location:")
    
    # FIND DESKTOP PATH and create a folder structure below it. 
    desktop = os.path.expanduser("~/Desktop")
    top_folder = 'SLA_AUTO_OUTPUT'
    filepath = os.path.join(desktop,'Current Items', 'SLA_Agenda', top_folder, month_dir)
    try:
        os.makedirs(filepath)
        print(filepath)
    except Exception as e: print(e)

    
    
    # Strip whitespace from left and right of column. Consider doing this for other columns     
    agenda_df['b_tradename'] = agenda_df.b_tradename.str.strip() 
    agenda_df['b_llc_name'] = agenda_df['b_llc_name'].str.strip()
    agenda_df['prim_address'] = agenda_df['prim_address'].str.strip()
    
    
    # 1 Make new folder path for each establishment
    # 2 This will be the primary address followed by a dash followed by the trade name if it exists, else the LLC name
    #      These will follow this pattern: '45 Avenue B - Lamias Fish Market'
    #
    for index, row in agenda_df.iterrows():
        est_filepath = ''
        if row.b_tradename != '':
            est_filepath = row.prim_address + ' - ' + row.b_tradename
        else:
            est_filepath = row.prim_address + ' - ' + row.b_llc_name

        fin_filepath = os.path.join(filepath, est_filepath)
        
        try:
            os.makedirs(fin_filepath)
            print(fin_filepath)
    
        except Exception as e: print(e)
        
        
    return agenda_df;

In [5]:
oct_pull =  r"C:\Users\MN03\Desktop\Current Items\SLA_Agenda\sla_oct.txt"

In [6]:
agenda_df = make_sla_folders(oct_pull)

Making SLA folders at following location:
C:\Users\MN03/Desktop\Current Items\SLA_Agenda\SLA_AUTO_OUTPUT\11-November 2021 SLA
C:\Users\MN03/Desktop\Current Items\SLA_Agenda\SLA_AUTO_OUTPUT\11-November 2021 SLA\509 E 6th St - Pineapple Club
C:\Users\MN03/Desktop\Current Items\SLA_Agenda\SLA_AUTO_OUTPUT\11-November 2021 SLA\6 St Marks Pl - JSG Saint Marks Singing LLC
C:\Users\MN03/Desktop\Current Items\SLA_Agenda\SLA_AUTO_OUTPUT\11-November 2021 SLA\22 Orchard St - Treasure Club Intl Inc
C:\Users\MN03/Desktop\Current Items\SLA_Agenda\SLA_AUTO_OUTPUT\11-November 2021 SLA\76 Forsyth St - Davids
C:\Users\MN03/Desktop\Current Items\SLA_Agenda\SLA_AUTO_OUTPUT\11-November 2021 SLA\119 Ave A - Superiority Burger
C:\Users\MN03/Desktop\Current Items\SLA_Agenda\SLA_AUTO_OUTPUT\11-November 2021 SLA\125 Rivington St - Corp to be formed by Brendan McElroy
C:\Users\MN03/Desktop\Current Items\SLA_Agenda\SLA_AUTO_OUTPUT\11-November 2021 SLA\135 Division St - Lucky Star
C:\Users\MN03/Desktop\Current Item

In [3]:
# Append automated output to Standard SLA Tracking Template
# https://stackoverflow.com/questions/57314254/pandas-df-write-to-excel-template-with-prepared-styles

TEMPLATE_PATH = r"C:\Users\MN03\Desktop\Calvin Docs\SLA\Automation Work\SLA_Tracker_Template.xlsx"

In [4]:
agenda_df

NameError: name 'agenda_df' is not defined

In [5]:
# Create new text file for emails to sla:
desktop = os.path.expanduser("~/Desktop")
top_folder = 'SLA_AUTO_OUTPUT'
filepath = os.path.join(desktop,'Current Items','SLA_Agenda')

sla_emails_text = os.path.join(filepath, 'sla_email_text.txt')

# In cases where the 'b_llc_name' column does not have a value, The b_tradename also contains the LLC name. 
# This should be fixed in the future, but for now the work around is to identify rows where both columns have values
# to identify where the 'b_tradename' column is truly a tradename. 

for index, row in agenda_df.iterrows():
    if (row.b_tradename != '' and row.b_llc_name != ''):
        with open(sla_emails_text, "a") as file:
            file.write("\n" + "\n" + "\n" + "CB3 Resolution re: " + row.b_tradename + " - " + row.prim_address + "\n"
                       "Re:    " + row.b_llc_name + "\n" + "       " + "d/b/a " + row.b_tradename + "\n" + "       " +
                       row.prim_address + "\n" + "       " + "New York, NY" + "\n")
            file.close() 
        
    else:
        with open(sla_emails_text, "a") as file:
            file.write("\n" + "\n" + "\n" + "CB3 Resolution re: " + row.b_tradename + " - " + row.prim_address +
                       "\n" + "Re:    " + row.b_tradename + "\n" + "       " +  row.prim_address + "\n" + "       " + 
                       "New York, NY" + "\n" )
            file.close() 

NameError: name 'agenda_df' is not defined

In [3]:
# Create new text file for EMAILS for admin approvals to sla:
desktop = os.path.expanduser("~/Desktop")
top_folder = 'SLA_AUTO_OUTPUT'
filepath = os.path.join(desktop,'Current Items','SLA_Agenda')

sla_emails_text = os.path.join(filepath, 'sla_email_text_admin_approvals.txt')

for index, row in agenda_df.iterrows():
        with open(sla_emails_text, "a") as file:
            file.write("\n \n \n" + "CB 3 No Objection To (New Application, Municipal Expansion, Alteration, Corporation Change) with stipulations, stipulations attached – " + row.prim_address + "\n \n" + """Please see the attached letter from CB 3 Manhattan stating no objection to the wine, beer,and cider application for """
                       + row.b_tradename + " located at " + row.prim_address + """, so long as the attached stipulations are included in the license agreement.""")
            file.close() 


NameError: name 'agenda_df' is not defined

In [5]:
# SLA APPLICATION TYPE OUTPUT
Nov_Path= r"C:\Users\MN03\Desktop\Current Items\SLA_Agenda\sla_app_type\sla_app_type.txt"

In [6]:
agenda_df = make_sla_folders(Nov_Path)

Making SLA folders at following location:
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\MN03/Desktop\\Current Items\\SLA_Agenda\\SLA_AUTO_OUTPUT\\11-November 2021 SLA'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\MN03/Desktop\\Current Items\\SLA_Agenda\\SLA_AUTO_OUTPUT\\11-November 2021 SLA\\167 Orchard St - Mother Duck'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\MN03/Desktop\\Current Items\\SLA_Agenda\\SLA_AUTO_OUTPUT\\11-November 2021 SLA\\6 Chatham Sq - Dim Sum Palace'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\MN03/Desktop\\Current Items\\SLA_Agenda\\SLA_AUTO_OUTPUT\\11-November 2021 SLA\\22 Orchard St - Treasure Club Intl Inc'
[WinError 183] Cannot create a file when that file already exists: 'C:\\Users\\MN03/Desktop\\Current Items\\SLA_Agenda\\SLA_AUTO_OUTPUT\\11-November 2021 SLA\\59 2nd Ave - Dim Sum Sam Inc'
[WinError 183] Cannot create a f

In [7]:
agenda_df

# Open the text file
#r+ = read/write access mode
agenda = open(Nov_Path, 'r+')
    
#Readlines creates a list, where each index contains a line of text, which in this case is a single establishment. 
contents = agenda.readlines()

# Close the text file
agenda.close()

print(contents)

['Alterations\n', '2.    Mother Duck (Orchard Flavor Partners), 167 Orchard St (alt/op/change method of operation: extend closing hours to 2am Sunday-Wednesday and 3am Thursday-Saturday)\n', 'New Liquor License Applications\n', '3.    Dim Sum Palace (Dim Sum V Inc), 6 Chatham Sq (op)\n', "4.    Treasure Club Int'l Inc, 22 Orchard St (North Store) (op)\n", '5.    Dim Sum Sam Inc, 59 2nd Ave (upgrade to op)\n', '6.    Entity to be formed by Eric Robinson, 67 1st Ave (wb)\n', '7.    Gala Dinners LLC, 85 Orchard St (op)\n', '8.    109 Brasserie LLC, 109 Ludlow St (op)\n', '9.    Nudibranch (Nudibranquio LLC), 125-127 1st Ave (op)\n', '10.    Corp to be formed by Brendon McElroy, 125 Rivington St (op)\n', '11.    Down & Out (Down & Out Brooklyn LLC), 503 E 6th St (op)\n', 'Items not heard at Committee\n', '12.    Moxy/East Village/Alphabet Bar & Cafe (US Dev Associates LLC & 11th Street Hospitality LLC), 112 E 11th St (alt/op/change method of operation: extends hours from 4pm-10pm all days 

In [140]:
  # This creates a dataframe where each row is a line from the agenda pull
agenda_df = pd.DataFrame(contents, columns=['line'])
    
    #Remove illegal character (apostrophe) from each string, and replace 'B'way' with 'Broadway'
agenda_df = agenda_df.apply(lambda x: x.str.replace("B'way", "Broadway"))
agenda_df = agenda_df.apply(lambda x: x.str.replace("'", ""))
    
    # This line creates new column that contains a bool series with 'True' for every line that starts with a a digit
    # Lines that do not contain a digit (agenda number) will be removed.
agenda_df['entry_row']= agenda_df['line'].str[0].str.isdigit()

In [141]:
agenda_df


# IDENTIFY SUBSTRINGS
# df[df['A'].str.contains("hello")]


# df.loc[(df.Event == 'Dance'),'Event']='Hip-Hop'

agenda_df.loc[agenda_df.line.str.contains("Alterations"), 'line']='Alteration' 

agenda_df.loc[agenda_df.line.str.contains("Items not heard at Committee"), 'line']='Item not heard at Committee' 

agenda_df.loc[agenda_df.line.str.contains("	Expansion onto Municipal Property"), 'line']='Expansion onto Municipal Property' 



In [142]:
# Iterate through every row. 
# If line does not start with a digit, take 'line' value and assign it to every row until there is another row without a digit. 
# see is_digit code above
# might also 

    # This line creates new column that contains a bool series with 'True' for every line that starts with a a digit
    # Lines that do not contain a digit (agenda number) will be removed.
   # agenda_df['entry_row']= agenda_df['line'].str[0].str.isdigit()

#initialize app_type column to blank string
agenda_df['app_type'] = ''

for index, row in agenda_df.iterrows():
    if row.line[0].isdigit() is False:
        a_type = row.line
        agenda_df.at[index, 'app_type'] = ''

    else:
        agenda_df.at[index, 'app_type'] = a_type
        
# Strip whitespace and newline characters from beginning and end of strings

agenda_df['app_type'] = agenda_df['app_type'].str.strip()

In [145]:
agenda_df

,line,entry_row,app_type
0,Alteration,False,
1,"2. Mother Duck (Orchard Flavor Partners), 1...",True,Alteration
2,New Liquor License Applications\n,False,
3,"3. Dim Sum Palace (Dim Sum V Inc), 6 Chatha...",True,New Liquor License Applications
4,"4. Treasure Club Intl Inc, 22 Orchard St (N...",True,New Liquor License Applications
5,"5. Dim Sum Sam Inc, 59 2nd Ave (upgrade to ...",True,New Liquor License Applications
6,"6. Entity to be formed by Eric Robinson, 67...",True,New Liquor License Applications
7,"7. Gala Dinners LLC, 85 Orchard St (op)\n",True,New Liquor License Applications
8,"8. 109 Brasserie LLC, 109 Ludlow St (op)\n",True,New Liquor License Applications
9,"9. Nudibranch (Nudibranquio LLC), 125-127 1...",True,New Liquor License Applications


In [113]:
def set_app_type(x):
    
    if x[0][0].isdigit() is False:
        index_dict.update({x.name: x.line})
        index_list.append(x.name)
        app_type = x.line
        print(app_type)
#         print(index_dict)
#         print("index list: " + str(index_list))
        
    else: 
        print(app_type)
        
    
        
        
# TO DO
# I have a dictionary with the index and app type description for all relevant rows. 
# Next I need to assign these to every column between the app type descriptor columns
# Get list of index - must be numerical for comparison
# For each app type description item in dictionary:
    # If there is an index that is higher
        # Set app type column to the app type description for all rows greater than index and less than next greatest index. 
    
    
    # If there is no higher index item:
        # Set 'app_type' column to the app type description for all rows greater than the index

        

        
#         app_type = app_type_init
#         print("app_type is: " + app_type) 
    
#        return ''

#         print(x)
        
#        return app_type
        
    
    
    

In [114]:
agenda_df["app_type"] = ''
index_dict = {}
index_list = []
agenda_df["app_type", ""] = agenda_df.apply(lambda x: set_app_type(x),axis=1)

Alteration


UnboundLocalError: local variable 'app_type' referenced before assignment

In [84]:
agenda_df["app_type"] = agenda_df.apply(lambda x: 'False' if x.line[0].isdigit() is False else 'True', axis=1)

In [ ]:
# new_df = df[df.apply(lambda x : bool_provider(x['Rev_M'],x['Year']),axis=1)]

In [ ]:
# Test pull changes from calvinbrown32 to cb3manhattan

In [1]:
# Pull test agenda (November) from personal github repo
import requests
url = "https://calvinbrown32.github.io/external_files/sample_agenda.txt"
nov_agenda_example = requests.get(url).content

In [2]:
nov_agenda_example

b"Alterations\n2.    Mother Duck (Orchard Flavor Partners), 167 Orchard St (alt/op/change method of operation: extend closing hours to 2am Sunday-Wednesday and 3am Thursday-Saturday)\nNew Liquor License Applications\n3.    Dim Sum Palace (Dim Sum V Inc), 6 Chatham Sq (op)\n4.    Treasure Club Int'l Inc, 22 Orchard St (North Store) (op)\n5.    Dim Sum Sam Inc, 59 2nd Ave (upgrade to op)\n6.    Entity to be formed by Eric Robinson, 67 1st Ave (wb)\n7.    Gala Dinners LLC, 85 Orchard St (op)\n8.    109 Brasserie LLC, 109 Ludlow St (op)\n9.    Nudibranch (Nudibranquio LLC), 125-127 1st Ave (op)\n10.    Corp to be formed by Brendon McElroy, 125 Rivington St (op)\n11.    Down & Out (Down & Out Brooklyn LLC), 503 E 6th St (op)\nItems not heard at Committee\n12.    Moxy/East Village/Alphabet Bar & Cafe (US Dev Associates LLC & 11th Street Hospitality LLC), 112 E 11th St (alt/op/change method of operation: extends hours from 4pm-10pm all days to 4pm-11pm all days and from roll-a-cover to fully 

In [3]:
sample_agenda = r.content

NameError: name 'r' is not defined